In [5]:
# ============================================================
# CONFIGURAÇÃO INICIAL E ETAPA 1 — EXTRAÇÃO DOS IDs (CSV)
# ============================================================

# URL base da API criada no MockAPI (App de Hábitos)
mock_api_url = "https://693ac0d79b80ba7262cb3aba.mockapi.io"

import pandas as pd

# ---- ETAPA 1: Leitura dos IDs dos usuários a partir do CSV ----

# Carrega o arquivo CSV contendo os IDs dos usuários
df = pd.read_csv("mock.csv")

# Converte a coluna 'UserID' em uma lista Python
user_ids = df["UserID"].tolist()

# Exibe os IDs extraídos para validação da etapa
print("IDs extraídos do CSV:", user_ids)


IDs extraídos do CSV: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [6]:
# ============================================================
# ETAPA 2 — EXTRAÇÃO DOS DADOS DOS USUÁRIOS VIA API
# ============================================================

import requests
import json


def get_user(user_id):
    """
    Recupera os dados de um usuário na API a partir do seu ID.
    Retorna None caso a requisição não seja bem-sucedida.
    """
    response = requests.get(f"{mock_api_url}/users/{user_id}")
    return response.json() if response.status_code == 200 else None


# Consulta os usuários listados no CSV e mantém apenas registros válidos
users = [
    user
    for user_id in user_ids
    if (user := get_user(user_id)) is not None
]

# Visualização dos dados extraídos (formato JSON)
print(json.dumps(users, indent=2))




[
  {
    "name": "Paulo",
    "primary_habit": "Exerc\u00edcio f\u00edsico",
    "frequency_per_week": 3,
    "goal": "criar consist\u00eancia",
    "insights": [
      {
        "habit": "Exerc\u00edcio f\u00edsico",
        "message": "Agende seus treinos como compromissos e crie um lembrete para n\u00e3o esquecer!",
        "frequency_per_week": 3
      },
      {
        "habit": "Exerc\u00edcio f\u00edsico",
        "message": "Agende os treinos como compromissos e use um lembrete para n\u00e3o esquecer!",
        "frequency_per_week": 3
      }
    ],
    "id": "1"
  },
  {
    "name": "Ana",
    "primary_habit": "Leitura",
    "frequency_per_week": 4,
    "goal": "Ler diariamente",
    "insights": [
      {
        "habit": "Leitura",
        "message": "Reserve 20 minutos antes de dormir para ler; fa\u00e7a disso uma rotina relaxante. \ud83d\udcda\u2728",
        "frequency_per_week": 4
      },
      {
        "habit": "Leitura",
        "message": "Crie um hor\u00e1rio fixo 

In [ ]:
# ============================================================
# ETAPA 3A — TRANSFORMAÇÃO DOS DADOS COM IA (GPT-4o-mini)
# ============================================================

from openai import OpenAI

# Defina sua chave da OpenAI apenas para execução local
openai_api_key = ""

if not openai_api_key:
    raise ValueError("Defina a variável openai_api_key com sua chave da OpenAI")


# Inicializa o cliente da OpenAI
client = OpenAI(api_key=openai_api_key)


def generate_ai_insight(user):
    """
    Gera um insight curto e personalizado para o usuário,
    considerando seu hábito principal e frequência semanal.
    """
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Você é um coach de hábitos que dá dicas práticas e realistas."
            },
            {
                "role": "user",
                "content": (
                    f"Crie uma dica curta (máximo de 100 caracteres) "
                    f"para ajudar {user['name']} a manter o hábito "
                    f"de {user['primary_habit']} "
                    f"{user['frequency_per_week']} vezes por semana."
                )
            }
        ]
    )

    return completion.choices[0].message.content


In [8]:
# ============================================================
# ETAPA 3B — APLICAÇÃO DA TRANSFORMAÇÃO NOS USUÁRIOS
# ============================================================

for user in users:
    insight = generate_ai_insight(user)

    print(f"Insight gerado para {user['name']}: {insight}")

    if "insights" not in user:
        user["insights"] = []

    user["insights"].append({
        "habit": user["primary_habit"],
        "message": insight,
        "frequency_per_week": user["frequency_per_week"]
    })

# Visualização final dos usuários após enriquecimento
print(json.dumps(users, indent=2))


Insight gerado para Paulo: Agende seus treinos como compromissos e não falhe! Marque no calendário e cumpra como prioridade.
Insight gerado para Ana: Defina um horário fixo na semana para leitura e mantenha-o como um compromisso inadiável!
Insight gerado para Carlos: Defina um lembrete diário e medite sempre no mesmo lugar para criar uma rotina mais fácil.
Insight gerado para Beatriz: Defina um horário fixo para estudar e use lembretes no celular para não esquecer!
Insight gerado para Lucas: Coloque um lembrete no celular para beber água 1x a cada hora que estiver acordado!
Insight gerado para Mariana: Crie uma rotina com horários fixos para dormir e acordar, até nos fins de semana!
Insight gerado para Rafael: Agende suas caminhadas como compromissos e use um lembrete no celular para não esquecer!
Insight gerado para Juliana: Planeje suas refeições e prepare lanches saudáveis no domingo para a semana! 🍏🥗
Insight gerado para Fernando: Defina dias e horários fixos para organizar; use um 

In [9]:
# ============================================================
# ETAPA 4 — LOAD DOS DADOS ENRIQUECIDOS NA API
# ============================================================

def update_user(user):
    """
    Atualiza os dados de um usuário na API por meio de
    uma requisição PUT, utilizando o ID como identificador.
    
    Retorna True em caso de sucesso (HTTP 200),
    ou False caso a atualização falhe.
    """
    response = requests.put(
        f"{mock_api_url}/users/{user['id']}",
        json=user
    )

    return response.status_code == 200


# ---- Execução do Load para todos os usuários ----
for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}")


User Paulo updated? True
User Ana updated? True
User Carlos updated? True
User Beatriz updated? True
User Lucas updated? True
User Mariana updated? True
User Rafael updated? True
User Juliana updated? True
User Fernando updated? True
User Camila updated? True
